## Tech Challeng FIAP

### Estudo de vinhos

#### O problema

Imagine agora, que você vai atuar como Expert em Data Analytics em uma empresa que exporta vinhos do Brasil para o mundo todo.

Sua área é recém-criada dentro da empresa, e você será responsável pelos relatórios iniciais a serem apresentados em uma reunião de investidores e acionistas, explicando a quantidade de vinhos exportados e os fatores externos que podem vir a surgir e que interferem nas análises:

1. Dados climáticos
2. Dados demográficos
3. Dados econômicos
4. Dados de avaliação de vinhos

O head de dados pediu para que você construísse uma tabela contendo as seguinte informações:

- País de origem (Brasil)
- País de destino
- Quantidade em litros de vinho exportado (utilize: 1KG = 1L)
- Valor em US$

Os dados que lhe forneceram são de uma vinícola parceira.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# carregando a base
base_vinhos = pd.read_csv(r'D:\Documentos\FIAP - Pos Tech Data Analytics\Tech Challeng - Vinhos\Dados\ExpVinho.csv', encoding='utf-8', sep=';')

In [3]:
display(base_vinhos.head(2))

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698


### Tratando os dados

In [4]:
# excluindo a coluna Id
base_vinhos = base_vinhos.drop('Id', axis=1)
base_vinhos = base_vinhos.set_index('País')

In [5]:
display(base_vinhos)

,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,1974,1974.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
País,,,,,,,,,,,,,,,,,,,,,
Afeganistão,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
África do Sul,0,0,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,0,0,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
Angola,0,0,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
Anguilla,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tuvalu,0,0,0,0,0,0,0,0,0,0,...,0,0,2,4,0,0,0,0,0,0
Uruguai,0,0,0,0,0,0,0,0,0,0,...,6180,18497,0,0,136774,149842,637117,997367,326093,454271
Vanuatu,0,0,0,0,0,0,0,0,0,0,...,0,0,18,31,0,0,0,0,0,0


Foi necessário criar dfs separados para vendas e quantidade, tratar um de cada vez e depois unificar

In [6]:
# separando as colunas de quantidade e valores
colunas = base_vinhos.columns

colunas_qtd = []
colunas_vendas = []


for coluna in colunas:
      if coluna.endswith('.1'):
            colunas_vendas.append(coluna)
      elif len(coluna) == 4:          
            colunas_qtd.append(coluna)
      else:
            pass
      
# criando dfs separados para as informações de quantidade e vendas
base_vinhos_qtd = base_vinhos[colunas_qtd]
base_vinhos_vendas = base_vinhos[colunas_vendas]

Tratando df de quantidade

In [7]:
# filtrando apenas os ultimos 15 anos
base_vinhos_qtd = base_vinhos_qtd.iloc[:, -16:]

# criando coluna de total
base_vinhos_qtd['Total'] = base_vinhos_qtd.sum(axis=1)

# removendo países sem dados de vendas
base_vinhos_qtd = base_vinhos_qtd.loc[base_vinhos_qtd['Total'] != 0]

# ordenando o df pela coluna de total
base_vinhos_qtd = base_vinhos_qtd.sort_values(by='Total', ascending=False)

# removendo a coluna de total porque pode atrapalhar em uma visualizaçao por exemplo
base_vinhos_qtd = base_vinhos_qtd.drop('Total', axis=1)

# pivotando as colunas para linhas
base_vinhos_qtd = base_vinhos_qtd.reset_index()
base_vinhos_qtd = base_vinhos_qtd.melt(id_vars=["País"], var_name="Periodo", value_name="Venda_volume")

# renomenado as colunas
base_vinhos_qtd.columns = ['Pais', 'Periodo', 'Venda_volume']

# transformando a coluna de período em data ano
base_vinhos_qtd['Periodo'] = base_vinhos_qtd['Periodo'].astype('Int64')

# transformando a coluna de quantidade em float
base_vinhos_qtd['Venda_volume'] = base_vinhos_qtd['Venda_volume'].astype(np.float64)

## todos os dados estao corretos
      # Pais -- str
      # Periodo -- Int64
      # Venda_volume -- float64

# -------- Exibindo a base depois dos tratamentos ----------

display(base_vinhos_qtd.head(5))

,Pais,Periodo,Venda_volume
0,Rússia,2008,6207658.0
1,Paraguai,2008,2191901.0
2,Estados Unidos,2008,443895.0
3,China,2008,8689.0
4,Haiti,2008,20.0


Tratando df de venda valor

In [8]:
# ajustando o nome das colunas do df
colunas = []

for coluna in base_vinhos_vendas.columns:
      coluna = coluna[:4]
      colunas.append(coluna)

base_vinhos_vendas.columns = colunas

# filtrando apenas os ultimos 15 anos
base_vinhos_vendas = base_vinhos_vendas.iloc[:, -16:]

# criando coluna de total
base_vinhos_vendas['Total'] = base_vinhos_vendas.sum(axis=1)

# removendo países sem dados de vendas
base_vinhos_vendas = base_vinhos_vendas.loc[base_vinhos_vendas['Total'] != 0]

# ordenando o df pela coluna de total
base_vinhos_vendas = base_vinhos_vendas.sort_values(by='Total', ascending=False)

# removendo a coluna de total porque pode atrapalhar em uma visualizaçao por exemplo
base_vinhos_vendas = base_vinhos_vendas.drop('Total', axis=1)

# pivotando as colunas para linhas
base_vinhos_vendas = base_vinhos_vendas.reset_index()
base_vinhos_vendas = base_vinhos_vendas.melt(id_vars=["País"], var_name="Periodo", value_name="Venda_volume")

# renomenado as colunas
base_vinhos_vendas.columns = ['Pais', 'Periodo', 'Venda_valor']

# transformando a coluna de período em data ano
base_vinhos_vendas['Periodo'] = base_vinhos_vendas['Periodo'].astype('Int64')

# transformando a coluna de quantidade em float
base_vinhos_vendas['Venda_valor'] = base_vinhos_vendas['Venda_valor'].astype(np.float64)

## todos os dados estao corretos
      # Pais -- str
      # Periodo -- Int64
      # Venda_valor -- float64

# -------- Exibindo a base depois dos tratamentos ----------

display(base_vinhos_vendas.head(5))

,Pais,Periodo,Venda_valor
0,Paraguai,2008,1374088.0
1,Rússia,2008,2352768.0
2,Estados Unidos,2008,804607.0
3,China,2008,25926.0
4,Reino Unido,2008,155076.0


Agora que temos dois dfs com informações distintas vamos unificar e criar apenas um df

In [9]:
# mesclando df
base_vinhos = pd.merge(base_vinhos_qtd, base_vinhos_vendas, how='inner', left_on=['Pais', 'Periodo'], right_on=['Pais', 'Periodo'])

# exibindo para ver como ficou o df
      # ------- Não rode esse código mais de uma vez para não mesclar em cima do df pronto ----------
display(base_vinhos.head(5))
display(base_vinhos.tail(5))

,Pais,Periodo,Venda_volume,Venda_valor
0,Rússia,2008,6207658.0,2352768.0
1,Paraguai,2008,2191901.0,1374088.0
2,Estados Unidos,2008,443895.0,804607.0
3,China,2008,8689.0,25926.0
4,Haiti,2008,20.0,20.0


,Pais,Periodo,Venda_volume,Venda_valor
2011,Martinica,2023,9.0,31.0
2012,Mauritânia,2023,0.0,0.0
2013,Macau,2023,7.0,6.0
2014,Toquelau,2023,3.0,10.0
2015,Tuvalu,2023,0.0,0.0


In [10]:
# Exportando o dataframe tratado
base_vinhos.to_csv(r'D:\Documentos\FIAP - Pos Tech Data Analytics\Tech Challeng - Vinhos\Dados\dados_exportacao_tratados.csv', index=False, sep=';') 